In [1]:
import sympy
import numpy as np
import dev_tools

In [2]:
x, y = sympy.symbols('x y')
m, n = sympy.symbols('m n', integer=True)
c = sympy.symbols('c')

In [20]:
polynomial = dev_tools.polynomials.get_random_polynomial(degree=4, max_coeff=10.)

In [21]:
p_monomial = c * x**m * y**n

In [26]:
p = 0.
for monomial in polynomial.monomials:
    p = p + p_monomial.subs([
        (m, monomial.x_exponent),
        (n, monomial.y_exponent),
        (c, monomial.coefficient)])
p

8.48369191687937*x**4 + 8.57494864809683*x**3*y - 9.51699421918885*x**3 + 7.87670886745937*x**2*y**2 - 3.5400422883021*x**2*y - 4.11307353523962*x**2 + 5.96326991897318*x*y**3 + 7.41971569860138*x*y**2 - 9.87132973959924*x*y - 1.71072760994778*x - 9.20459509142649*y**4 - 1.82203390157793*y**3 - 9.83696088677537*y**2 - 6.46052782104034*y - 1.06365113561688

$$
\int_{\Phi(\hat{K})} f(x) dx
=
\int_{\hat{K}} f(\Phi(\hat{x})) \mathrm{det} D\Phi d\hat{x}
$$

$$
\Phi(\hat{x}, \hat{y}) = r_1 + \hat{x} (r_2 - r_1) + \hat{y} (r_3 - r_1)
$$

In [6]:
r_1x, r_1y = sympy.symbols('r_1x r_1y')
r_2x, r_2y = sympy.symbols('r_2x r_2y')
r_3x, r_3y = sympy.symbols('r_3x r_3y')
x_hat, y_hat = sympy.symbols('x_hat y_hat')

In [7]:
Phi = sympy.Matrix([
    r_1x + x_hat*(r_2x - r_1x) + y_hat*(r_3x - r_1x),
    r_1y + x_hat*(r_2y - r_1y) + y_hat*(r_3y - r_1y),
])
Phi

Matrix([
[r_1x + x_hat*(-r_1x + r_2x) + y_hat*(-r_1x + r_3x)],
[r_1y + x_hat*(-r_1y + r_2y) + y_hat*(-r_1y + r_3y)]])

In [8]:
jacobian = Phi.jacobian(sympy.Matrix([x_hat, y_hat]))

In [9]:
det_Phi = sympy.det(jacobian)

In [10]:
det_Phi

r_1x*r_2y - r_1x*r_3y - r_1y*r_2x + r_1y*r_3x + r_2x*r_3y - r_2y*r_3x

In [15]:
def integrate_on_triangle(f: sympy.core.add.Add, coordinates: np.ndarray) -> float:
    """
    f: a function depending on symbols x, y, i.e. f(x, y)
    coordinates: numerical values of the triangle's vertices
    """
    r1 = coordinates[0, :]
    r2 = coordinates[1, :]
    r3 = coordinates[2, :]

    integrand = f.subs([
        (x, Phi[0]),
        (y, Phi[1])
    ])

    result = sympy.integrate(integrand, (y_hat, 1-x_hat, 1), (x_hat, 0, 1)) * det_Phi

    numerical_result = result.subs([
        (r_1x, r1[0]),
        (r_1y, r1[1]),
        (r_2x, r2[0]),
        (r_2y, r2[1]),
        (r_3x, r3[0]),
        (r_3y, r3[1])
    ]).evalf()

    return float(numerical_result)

In [13]:
k = sympy.symbols('k')
result = sympy.integrate(k, (y, 0, 1-x), (x, 0, 1))
result.subs(k, 0.5).evalf()

0.250000000000000

In [24]:
integrate_on_triangle(f=p, coordinates=dev_tools.generate_random_triangle())

-1.4891480669410093